In [5]:
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI, RateLimitError
import os

load_dotenv()

deployment_name='gpt4Test'
max_tokens=4096

client = AzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),  
    api_version=os.getenv("AZURE_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT")
)

def generate_chat_gpt_query(prompt):
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
        n=1,
        stop=None
    )

    if response.choices:
        return response.choices[0].message.content.strip()
    else:
        return "No suggestion generated."
    
df = pd.read_csv("dataset_with_sentiment.csv")

df_negative = df[df['label'] == 'positive']

specific_days = ['2023-01-02', '2023-01-13', '2023-01-27', '2023-02-27', '2023-03-10', '2023-03-24', '2023-03-24', '2023-04-07', '2023-04-21', '2023-05-05', '2023-05-05', '2023-05-19', '2023-06-02', '2023-06-16']

df_specific_days = df_negative[df_negative['day'].isin(specific_days)].drop(columns=["Unnamed: 0"])

context = df_specific_days.to_json(orient="records")

query = (
    "Given a context containing positive feedbacks from different sprint retrospectives grouped by days, I would like insights to assist the team manager in making performance reviews for team members. Please analyze the feedbacks and provide:\n"
    "1. Personalized feedback about each team member mentioned in feedbacks.\n"
    "\tAnswer should be in the following format:\n"
    "\t**Person's name**\n"
    "\tList of all feedbacks where this person is mentioned in format: ```day``` - ```feedback from 'text' column```\n"
    "\tYour conclusion about the person"
)

prompt = "Context:\n" + context + "\n\nPrompt:\n" + query
print(prompt)
ai_response = generate_chat_gpt_query(prompt)
print(ai_response)

# Main Topics:
# 1. Team Collaboration and Support (e.g., helping each other, teamwork, cooperation on tasks)
# 2. Achievements and Successes (e.g., sprint goals achieved, big releases, successful migrations)
# 3. Individual Contributions and Recognition (e.g., specific people being thanked or recognized for their contributions)
# 4. Process Improvements and Adaptations (e.g., adopting new tools like Jira, refining tasks together, effective meetings)


Context:
[{"day":"2023-01-02","text":"deliver the goal of sprint","human_label":"positive","label":"positive"},{"day":"2023-01-02","text":"continue to help each other","human_label":"positive","label":"positive"},{"day":"2023-01-02","text":"celebrate success -  pri trial launch - empty lane on demo","human_label":"positive","label":"positive"},{"day":"2023-01-02","text":"sprint goal achieved!!!","human_label":"positive","label":"positive"},{"day":"2023-01-02","text":"follow the person who is talking","human_label":"neutral","label":"positive"},{"day":"2023-01-02","text":"record knowledge sharing sessions","human_label":"neutral","label":"positive"},{"day":"2023-01-13","text":"andrei, great job  with helping  customer \ud83d\udc4f","human_label":"positive","label":"positive"},{"day":"2023-01-13","text":"nice daily format like the idea with coloring tasks with goals colors","human_label":"positive","label":"positive"},{"day":"2023-01-13","text":"quick response when asking for review","hu